In [7]:
# TODO:

# From Openstreetmap:
# - What it provides: Distance to city centers, shopping hubs, or transport nodes (e.g., train stations).
# - Population -   "gis_osm_places_free_1"
# - Distance from water - gis_osm_waterways_free_1
# - Distance from transport - gis_osm_transport_a_free_1

# After I process OpenStreetMaps:

# - Use your existing dataset of store locations.
#     - Calculate the density of similar retailers (e.g., supermarkets) within a 1 km radius using geospatial libraries like geopandas or scipy.
# - I want to calculate how the size of each location i.e. x metres squared 
# - Business rates for each location - https://www.data.gov.uk/dataset/f027145b-b55f-4602-b889-b28a8ca04462/business-rates



In [1]:
import pandas as pd
import geopandas as gpd
import utils
import numpy as np


pd.set_option('display.max_columns', None)

Reasonings:
- Overture, AllThePlaces basically only give us the location and type of location (overture). Which is now irrelevant due to the Geolytix dataset
- Geolytix dataset contains retail stored and we have filtered it to the specified region
- Geolytix address is more accurate than OpenLocal so we are using that.

Ideas:
- Once we have a starting point with the dataset, we will subset to 
- Tesco/Sainsburys - get turnover per sq ft


Data sources:
- ONS Postcode LSOA look up - https://geoportal.statistics.gov.uk/datasets/e7824b1475604212a2325cd373946235/about
- ONS small area GVA estimates - https://www.ons.gov.uk/economy/grossvalueaddedgva/datasets/uksmallareagvaestimates
- ONS Population - https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationandmigrationstatisticstransformationlowerlayersuperoutputareapopulationdifferenceenglandandwales
- ONS Household income - https://www.ons.gov.uk/economy/regionalaccounts/grossdisposablehouseholdincome/datasets/ukgrossdisposablehouseholdincomegdhiforothergeographicareas
- Geolytix - https://drive.google.com/file/d/1B8M7m86rQg2sx2TsHhFa2d-x-dZ1DbSy/view
- OpenLocal - https://drive.google.com/drive/folders/1q2LJ-5U41rxGPUpirC3i0QLhiYVaixlR
- OpenStreetMap - https://download.geofabrik.de/europe/united-kingdom/england/greater-manchester.html

Definitions:
GVA: Gross Value Added
TTWA: Travel to Work Areas
GDHI: Gross Disposable Household Income
Town: Towns and cities
WGPC: WEstminster Government Parliamentary Constituency
CCG: CLinical Commissioning Groups


In [2]:
gdf = gpd.read_file("data/raw/openlocal/GeoTAM_Hackathon_OpenLocal.gpkg")
gdf["longitude"] = round(gdf.geometry.x, 6)
gdf["latitude"] = round(gdf.geometry.y, 6)

gdf = gdf[[
    'geometry', 'longitude', 'latitude',
    'voapropertyaddress', 'voapropertytown','voapropertycounty', 'voapropertypostcode', 'voapostcodecentroid',
    "voabillingauthoritycode", "voascatcode", "voascatname", 'voafloorarea', 'voarateablevalue', 
    'laratespaid', 'laratesreliefs', 
]]

min_lon, max_lon, min_lat, max_lat = gdf.longitude.min(), gdf.longitude.max(), gdf.latitude.min(), gdf.latitude.max()
# overture_df = utils.extract_overture_data(min_lon, max_lon, min_lat, max_lat)
# atp_df = utils.extract_all_the_places_data(min_lon, max_lon, min_lat, max_lat)

geolytix_df = pd.read_csv("data/raw/geolytix/geolytix_retailpoints_v33_202408.csv").rename(columns={"long_wgs":"longitude", "lat_wgs":"latitude"})
geolytix_df = geolytix_df[
    (geolytix_df["longitude"] >= min_lon) & (geolytix_df["longitude"] <= max_lon) & (geolytix_df["latitude"] >= min_lat) & (geolytix_df["latitude"] <= max_lat)
]
geolytix_df["longitude"] = round(geolytix_df["longitude"], 6)
geolytix_df["latitude"] = round(geolytix_df["latitude"], 6)



In [ ]:
geolytix_df["add_one"] = geolytix_df["add_one"].apply(lambda x: x.upper())
gdf["add_one"] = gdf["voapropertyaddress"].apply(lambda x: x.split(',')[0])

retail_df = geolytix_df.merge(
    gdf.drop(["geometry", "longitude", "latitude", 'voapropertyaddress', 'voapropertytown','voapropertycounty', 'voapropertypostcode', 'voapostcodecentroid'], axis=1), 
    on=["add_one"], how="left")
retail_df = retail_df.replace(r'^\s*$', np.nan, regex=True)
def count_non_null(row):
    return row.notnull().sum()
retail_df['non_null_count'] = retail_df.apply(count_non_null, axis=1)
retail_df = retail_df.sort_values(by=['id', 'non_null_count'], ascending=[True, False])
retail_df = retail_df.drop_duplicates(subset=['id'], keep='first')

df = utils.using_ons_datasets(retail_df)


In [6]:
df[df.id == 1010003132]


,id,retailer,fascia,store_name,add_one,add_two,town,suburb,postcode,longitude,latitude,bng_e,bng_n,pqi,open_date,size_band,county,voabillingauthoritycode,voascatcode,voascatname,voafloorarea,voarateablevalue,laratespaid,laratesreliefs,non_null_count,lsoa,gva_millions,cencus_pop,ttwa_code,town_code,pc_code,ward_code,ccg_code,ttwa_gdhi_millions,town_gdhi_millions,wgpc_gdhi_millions,ward_gdhi_millions,ccg_gdhi_millions
123,1010003132,Sainsburys,Sainsburys,Sainsburys Ashton Moss,2 LORD SHELDON WAY,NaN,Ashton Under Lyne,Waterloo,OL6 7TE,-2.105918,53.491396,393070.001028,399414.998877,Rooftop,NaN,"30,138 ft2 > (2,800 m2)",Greater Manchester,E08000008,139,Hypermarkets/Superstores (over 2500m^2),7097.41,1240000.0,677040.0,NaN,21,E01005948,418.249581,2245.0,E30000239,E35000787,E14000537,E05000816,E38000182,54907.789249,768.690987,1657.648548,193.535906,4823.099348


In [ ]:
# Resuming with Openstreetmap data... (see above for task list)

In [131]:
gpd.read_file("data/raw/openstreetmap/gis_osm_railways_free_1.shp")

,osm_id,code,fclass,name,layer,bridge,tunnel,geometry
0,928999,6101,rail,Liverpool to Manchester Line,0,F,F,"LINESTRING (-2.28446 53.48026, -2.29499 53.481..."
1,929904,6101,rail,Liverpool to Manchester Line,0,F,F,"LINESTRING (-2.2918 53.46196, -2.29249 53.4618..."
2,929905,6102,light_rail,Metrolink,0,F,F,"LINESTRING (-2.2794 53.46058, -2.27997 53.46022)"
3,3663332,6102,light_rail,None,0,F,F,"LINESTRING (-2.23821 53.4818, -2.23817 53.4817..."
4,3996086,6101,rail,West Coast Main Line,0,F,F,"LINESTRING (-2.60031 53.46043, -2.60053 53.460..."
...,...,...,...,...,...,...,...,...
3892,1319220899,6101,rail,Hope Valley Line Rosehill Branch,0,F,F,"LINESTRING (-2.06548 53.40654, -2.0657 53.4066..."
3893,1319220900,6101,rail,Hope Valley Line Rosehill Branch,0,F,F,"LINESTRING (-2.06584 53.40683, -2.06571 53.406..."
3894,1319220901,6101,rail,Hope Valley Line Rosehill Branch,0,F,F,"LINESTRING (-2.06719 53.40319, -2.0668 53.4034..."
3895,1319223135,6101,rail,Hope Valley Line,0,F,F,"LINESTRING (-2.11232 53.42372, -2.1115 53.42343)"


In [5]:
overture_df["categories_primary"] = overture_df.categories.fillna("Unknown").apply(lambda x: x["primary"] if isinstance(x, dict) else "Unknown")

In [15]:
tesco_overture = overture_df[overture_df.names_primary.str.startswith("TESCO")]
print(tesco_overture.shape)
tesco_atp = atp_df[atp_df.spider_id.str.startswith("tesco")]
print(tesco_atp.shape)
tesco_test = tesco_atp.merge(tesco_overture[["names_primary", "longitude", "latitude", "categories", "categories_primary"]], on=["longitude", "latitude"],  how="left")

(220, 11)
(162, 12)


In [29]:
tesco_overture[
    (tesco_overture["longitude"] == -2.425578)
    & (tesco_overture["latitude"] == 53.418544)
     
]

,id,names_primary,address,city,state,postcode,geometry,longitude,latitude,categories,categories_primary
1169,08f1951a059142a103e72af6e55ccedc,TESCO EXPRESS,CENTRAL RD,PARTINGTON,None,M31 4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",-2.425578,53.418544,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket


In [33]:
tesco_test[tesco_test["postcode"].str.startswith("M31")]
tesco_test

,longitude,latitude,spider_id,shop,full_address,address,city,postcode,business_name,branch,website,brand,names_primary,categories,categories_primary
0,-2.353317,53.517281,tesco_gb,convenience,NaN,Former Brook Tavern 656 Manchester Rd,NaN,M27 9RA,Tesco Express,Manchester Swinton,https://www.tesco.com/store-locator/worsley/fo...,Tesco Express,TESCO EXPRESS,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket
1,-2.205895,53.349625,tesco_gb,NaN,NaN,Kiln Croft Lane Handforth,NaN,SK9 3PA,Tesco Pharmacy,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/t...,Tesco,NaN,NaN,NaN
2,-2.202831,53.349394,tesco_gb,NaN,NaN,"Tesco Stores Ltd, Kiln Croft Lane Handforth",NaN,SK9 3PA,Tesco,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/t...,Tesco,NaN,NaN,NaN
3,-2.205893,53.349615,tesco_gb,supermarket,NaN,"Tesco Stores Ltd, Kiln Croft Lane Handforth",NaN,SK9 3PA,Tesco Extra,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/t...,Tesco Extra,NaN,NaN,NaN
4,-2.205924,53.349493,tesco_gb,NaN,NaN,Kiln Croft Lane Handforth,NaN,SK9 3PA,Tesco Café,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/k...,Tesco,TESCO CAFE,"{'primary': 'shopping_center', 'alternate': ['...",shopping_center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-2.347290,53.385508,tesco_gb,NaN,NaN,Manor Road,NaN,WA15 9QT,Tesco Pharmacy,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco,TESCO PHARMACY,"{'primary': 'pharmacy', 'alternate': ['drugsto...",pharmacy
158,-2.347339,53.385499,tesco_gb,NaN,NaN,Manor Road,NaN,WA15 9QT,Tesco Café,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco,TESCO CAFE,"{'primary': 'cafe', 'alternate': ['restaurant']}",cafe
159,-2.347299,53.385554,tesco_gb,supermarket,NaN,Manor Road,NaN,WA15 9QT,Tesco Extra,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco Extra,NaN,NaN,NaN
160,-2.311650,53.392414,tesco_gb,convenience,NaN,57 - 59 Briarfield Road Timperley,NaN,WA15 7DD,Tesco Express,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco Express,TESCO EXPRESS,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket


In [31]:
gdf[gdf["voapropertypostcode"].str.startswith("M31 4EL")]

,_uid_,ogc_fid,voabillingauthoritycode,voabillingauthorityname,voabillingauthorityrefcode,voauarn,labillingreferencecode,laratepayername,laaccountstartdate,larecorddate,laoccupationstate,laoccupationstatedate,laratespaid,laratesreliefs,laratesreliefsamount,laratesreliefsdate,voapropertyaddress,voapropertytown,voapropertycounty,voapropertypostcode,voapostcodecentroid,lacorrespondenceaddress,voascatcode,voascatname,voasubcategory,voacategory,voastartdate,voafloorarea,voarateablevalue,laupdatecycle,voaepoch,geometry,longitude,latitude
17249,17250,2755665,E08000009,Trafford,,196423116,22630675,,,,,,,,,,"SHOP A SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,54.1,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17250,17251,2754343,E08000009,Trafford,,76719116,22630350,,,,,,,,,,"6 SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,50.9,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17251,17252,2754344,E08000009,Trafford,,76720116,22630375,,,,,,,,,,"7 SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,52.5,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17252,17253,2754345,E08000009,Trafford,,76721116,22630400,,,,,,,,,,"8 SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,55.1,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17253,17254,2754329,E08000009,Trafford,,76698116,22630725,,,,,,,,,,"SHOP C SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,024,Betting Offices,RETAIL_FINANCIAL_AND_PROFESSIONAL_SERVICES,RETAIL,2011-03-22,48.8,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17254,17255,2754330,E08000009,Trafford,,76704116,22630550,,,,,,,,,,"15 SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,021,Banks/Insurance/Building Society Offices & Oth...,RETAIL_FINANCIAL_AND_PROFESSIONAL_SERVICES,RETAIL,2014-06-17,144.33,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17255,17256,2754331,E08000009,Trafford,,76705116,22630125,,,,,,,,,,"1 SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,55.8,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17256,17257,2754332,E08000009,Trafford,,76706116,22630450,,,,,,,,,,"10/10A SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,254.03,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17257,17258,2754333,E08000009,Trafford,,76707116,22630475,,,,,,,,,,"11 AND 12 SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,205.1,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873
17258,17259,2754334,E08000009,Trafford,,76708116,22630500,,,,,,,,,,"13 SHOPPING CENTRE, MANCHESTER, M31 4EL",MANCHESTER,,M31 4EL,0101000000425e0f26c56703c0b51666a19db54a40,,249,Shops,RETAIL_HIGH_STREET,RETAIL,2011-03-22,87.5,,,2010-0046-0000,POINT (-2.42567 53.41887),-2.425669,53.418873


In [21]:
tesco_test.merge(
    gdf[["longitude", "latitude", "voapropertypostcode", "voascatcode", "voascatname", "voafloorarea", "voarateablevalue", "laratespaid"]],
    on=["longitude", "latitude"],  how="left"
)

,longitude,latitude,spider_id,shop,full_address,address,city,postcode,business_name,branch,website,brand,names_primary,categories,categories_primary,voapropertypostcode,voascatcode,voascatname,voafloorarea,voarateablevalue,laratespaid
0,-2.353317,53.517281,tesco_gb,convenience,NaN,Former Brook Tavern 656 Manchester Rd,NaN,M27 9RA,Tesco Express,Manchester Swinton,https://www.tesco.com/store-locator/worsley/fo...,Tesco Express,TESCO EXPRESS,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket,NaN,NaN,NaN,NaN,NaN,NaN
1,-2.205895,53.349625,tesco_gb,NaN,NaN,Kiln Croft Lane Handforth,NaN,SK9 3PA,Tesco Pharmacy,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/t...,Tesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.202831,53.349394,tesco_gb,NaN,NaN,"Tesco Stores Ltd, Kiln Croft Lane Handforth",NaN,SK9 3PA,Tesco,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/t...,Tesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-2.205893,53.349615,tesco_gb,supermarket,NaN,"Tesco Stores Ltd, Kiln Croft Lane Handforth",NaN,SK9 3PA,Tesco Extra,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/t...,Tesco Extra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-2.205924,53.349493,tesco_gb,NaN,NaN,Kiln Croft Lane Handforth,NaN,SK9 3PA,Tesco Café,Wilmslow Handforth,https://www.tesco.com/store-locator/wilmslow/k...,Tesco,TESCO CAFE,"{'primary': 'shopping_center', 'alternate': ['...",shopping_center,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-2.311650,53.392414,tesco_gb,convenience,NaN,57 - 59 Briarfield Road Timperley,NaN,WA15 7DD,Tesco Express,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco Express,TESCO EXPRESS,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket,WA15 7DD,409,Cafes,67.55,4900.0,
164,-2.311650,53.392414,tesco_gb,convenience,NaN,57 - 59 Briarfield Road Timperley,NaN,WA15 7DD,Tesco Express,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco Express,TESCO EXPRESS,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket,WA15 7DD,018,ATMs,1.0,,
165,-2.311650,53.392414,tesco_gb,convenience,NaN,57 - 59 Briarfield Road Timperley,NaN,WA15 7DD,Tesco Express,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco Express,TESCO EXPRESS,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket,WA15 7DD,249,Shops,207.96,,
166,-2.311650,53.392414,tesco_gb,convenience,NaN,57 - 59 Briarfield Road Timperley,NaN,WA15 7DD,Tesco Express,Altrincham,https://www.tesco.com/store-locator/altrincham...,Tesco Express,TESCO EXPRESS,"{'primary': 'supermarket', 'alternate': ['reta...",supermarket,WA15 7DD,249,Shops,338.77,16750.0,


In [11]:
tesco_test[tesco_test.postcode_x.str.startswith("M31")]

,longitude,latitude,spider_id,shop,full_address,address_x,city_x,postcode_x,business_name,branch,website,brand,id,names_primary,address_y,city_y,state,postcode_y,geometry,categories,categories_primary
39,-2.425578,53.418544,tesco_gb,convenience,NaN,Central Rd,NaN,M31 4EL,Tesco Express,Central Rd,https://www.tesco.com/store-locator/partington...,Tesco Express,08f1951a059142a103e72af6e55ccedc,TESCO EXPRESS,CENTRAL RD,PARTINGTON,None,M31 4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","{'primary': 'supermarket', 'alternate': ['reta...",supermarket
